In [1]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import time 
import pandas as pd
import json
import re

In [2]:
hh_main_link = 'https://hh.ru/'
sj_main_link = 'https://russia.superjob.ru'
header = {'Accept': '*/*',
          'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0'}
proxi = {'http': ''}
search_str = 'python'
first_page = 0

In [3]:
def hh(hh_main_link, search_str, first_page):
    data = list()
    html = requests.get(f'{hh_main_link}search/vacancy?L_is_autosearch=false&area=1&clusters=true&enable_snippets=true&text={search_str}&page={first_page}', headers=header, proxies=proxi).text
    parsed_html = bs(html, 'lxml')
    #Находим номер последней страницы 
    last_page = int(parsed_html.find_all(attrs={'class':'bloko-button HH-Pager-Control'})[-1]['data-page'])

    
    for i in range(last_page+1): 
        vacancy_block = parsed_html.find('div',{'class':'vacancy-serp'} )
        vacancy_list = vacancy_block.findChildren(recursive=False)
        for vacancy in vacancy_list:
            vacancy_dict={}
            a = vacancy.find('a',{'class':'bloko-link HH-LinkModifier'})
            if a != None:
                vacancy_dict['site'] = 'HH'
                vacancy_dict['name'] = a.getText()
                vacancy_dict['link'] = a['href']
                compensation = vacancy.find('div', {'class': 'vacancy-serp-item__sidebar'}).text.replace('\xa0', ' ')
                if not compensation:
                    vacancy_dict['compensation_min'] = 'По договоренности'
                    vacancy_dict['compensation_max'] = ' '
                else:
                    compensation = compensation.replace('\xa0', ' ').split('-')
                    compensation_min=compensation[0]
                    vacancy_dict['compensation_min'] =compensation_min
                    if len(compensation)>1:
                        compensation_max=compensation[1]
                        vacancy_dict['compensation_max'] = compensation_max
                    else:
                        compensation_max= ' '
                        vacancy_dict['compensation_max'] = compensation_max
                    
                data.append(vacancy_dict)
        
        first_page +=1
        time.sleep(1)
        html = requests.get(f'{hh_main_link}search/vacancy?L_is_autosearch=false&area=1&clusters=true&enable_snippets=true&text={search_str}&page={first_page}', headers=header, proxies=proxi).text
        parsed_html = bs(html, 'lxml')
    return data

In [4]:
def sj(sj_main_link, search_str):
    data_sj=list()
    html = requests.get(f'{sj_main_link}/vacancy/search/?keywords={search_str}&geo[c][0]=1', headers=header, proxies=proxi).text
    parsed_sj = bs(html, 'lxml')
    while parsed_sj:
        vacancy_block = parsed_sj.find('div', {'style':"display:block"})
        vacancy_list = vacancy_block.findChildren('div', {'class':'_3zucV _2GPIV f-test-vacancy-item i6-sc _3VcZr'}, recursive=False)
                
        for vacancy in vacancy_list:
            vacancy_dict={}
            b = vacancy.find('a', {'class':'_1QIBo'}) 
            vacancy_dict['site'] = 'SJ'
            vacancy_dict['name'] = b.text
            vacancy_dict['link'] = sj_main_link+b['href']
           
            compensation_block = vacancy.find('span', {'class':'_3mfro _2Wp8I f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'})
            compensation_sj = compensation_block.text.replace('\xa0', ' ')
          
            if compensation_sj == 'По договорённости':
                vacancy_dict['compensation_min'] = 'По договорённости'
                vacancy_dict['compensation_max'] = ' '
            
            else:        
                pattern_number = re.compile('[0-9]{1,3}\s{1}[0-9]{3}')
                pattern_ot = re.compile('от')
                pattern_do = re.compile('до')
                number = re.findall(pattern_number, compensation_sj)
         
                if len(number) == 2:
                    vacancy_dict['compensation_min'] = number[0]
                    vacancy_dict['compensation_max'] = number[1]
                elif len(number) == 1:
                    ot = re.findall(pattern_ot, compensation_sj)
                    do = re.findall(pattern_do, compensation_sj)
                    if not do:
                        vacancy_dict['compensation_min'] = number[0]
                        vacancy_dict['compensation_max'] = '-'
                    elif not ot:
                        vacancy_dict['compensation_min'] = '-'
                        vacancy_dict['compensation_max'] = number[0]
            data_sj.append(vacancy_dict)
        next_page_search = parsed_sj.find("a", {'class':"icMQ_ _1_Cht _3ze9n f-test-button-dalshe f-test-link-Dalshe"})
        if next_page_search:
            html=requests.get(sj_main_link+next_page_search['href'], headers=header, proxies=proxi).text
            parsed_sj = bs(html, 'lxml')
                
        else:
            parsed_sj = None
    return (data_sj)

In [5]:
result_sj = sj(sj_main_link, search_str)
len(result_sj)


60

In [6]:
result_hh = hh(hh_main_link, search_str,first_page)
len(result_hh)

2000

In [7]:
with open("Lesson2_result.json", "w", encoding="utf-8") as file:
    json.dump(result_sj, file)
with open("Lesson2_result.json", "w", encoding="utf-8") as file:
    json.dump(result_hh, file)

In [8]:
df_sj = pd.DataFrame(result_sj)
df_sj

,site,name,link,compensation_min,compensation_max
0,SJ,Разработчик ИСУ (SQL/Python),https://russia.superjob.ru/vakansii/razrabotch...,По договорённости,
1,SJ,Программист Python,https://russia.superjob.ru/vakansii/programmis...,По договорённости,
2,SJ,Разработчик Python,https://russia.superjob.ru/vakansii/razrabotch...,170 000,200 000
3,SJ,Педагог дополнительного образования (со знание...,https://russia.superjob.ru/vakansii/pedagog-do...,По договорённости,
4,SJ,Разработчик Python,https://russia.superjob.ru/vakansii/razrabotch...,35 000,-
5,SJ,Программист Python/C++,https://russia.superjob.ru/vakansii/programmis...,По договорённости,
6,SJ,Инженер-программист АСУТП,https://russia.superjob.ru/vakansii/inzhener-p...,По договорённости,
7,SJ,Заместитель руководителя ИТ-дирекции,https://russia.superjob.ru/vakansii/zamestitel...,100 000,120 000
8,SJ,Системный администратор,https://russia.superjob.ru/vakansii/sistemnyj-...,30 000,-
9,SJ,Веб-разработчик / Веб-программист,https://russia.superjob.ru/vakansii/veb-razrab...,По договорённости,


In [9]:
df_hh = pd.DataFrame(result_hh)
df_hh

,site,name,link,compensation_min,compensation_max
0,HH,Программист-разработчик python (midle),https://spb.hh.ru/vacancy/35758988?query=python,90 000,200 000 руб.
1,HH,Fullstack-разработчик (Django (python) + React),https://spb.hh.ru/vacancy/36202473?query=python,150 000,200 000 руб.
2,HH,"C# техлид (back-end, teamlead)",https://spb.hh.ru/vacancy/36317669?query=python,от 250 000 руб.,
3,HH,C# разработчик (начинающий/миддл),https://spb.hh.ru/vacancy/35857760?query=python,80 000,140 000 руб.
4,HH,Программист-разработчик (React \ ExtJS),https://spb.hh.ru/vacancy/35763109?query=python,По договоренности,
...,...,...,...,...,...
1995,HH,Compiler Software Engineer,https://spb.hh.ru/vacancy/35643706?query=python,По договоренности,
1996,HH,Ведущий системный администратор,https://spb.hh.ru/vacancy/36238051?query=python,По договоренности,
1997,HH,Data supervisor (Business analyst),https://spb.hh.ru/vacancy/35282839?query=python,По договоренности,
1998,HH,Тестировщик,https://spb.hh.ru/vacancy/36216624?query=python,По договоренности,


In [10]:
df=df_hh.append(df_sj)
df

,site,name,link,compensation_min,compensation_max
0,HH,Программист-разработчик python (midle),https://spb.hh.ru/vacancy/35758988?query=python,90 000,200 000 руб.
1,HH,Fullstack-разработчик (Django (python) + React),https://spb.hh.ru/vacancy/36202473?query=python,150 000,200 000 руб.
2,HH,"C# техлид (back-end, teamlead)",https://spb.hh.ru/vacancy/36317669?query=python,от 250 000 руб.,
3,HH,C# разработчик (начинающий/миддл),https://spb.hh.ru/vacancy/35857760?query=python,80 000,140 000 руб.
4,HH,Программист-разработчик (React \ ExtJS),https://spb.hh.ru/vacancy/35763109?query=python,По договоренности,
...,...,...,...,...,...
55,SJ,Backend (PHP),https://russia.superjob.ru/vakansii/backend-33...,По договорённости,
56,SJ,Педагог дополнительного образования (центр циф...,https://russia.superjob.ru/vakansii/pedagog-do...,По договорённости,
57,SJ,"Программист, инженер-разработчик",https://russia.superjob.ru/vakansii/programmis...,50 000,80 000
58,SJ,Системный администратор Linux,https://russia.superjob.ru/vakansii/sistemnyj-...,-,70 000
